In [1]:
%cd '..'

/mnt/scratch/l_singh/hdb/project_dir/histonedb/CURATED_SET


In [2]:
import inspect
import os
import re
import sys

import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((561, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs.data[cs.data["accession"] == "AAO24603.1"]

Empty DataFrame
Columns: [accession, type, variant_group, variant, doublet, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, references, sequence]
Index: []

In [6]:
cs.data.head()

accession type variant_group         variant doublet  \
NP_505463.1        NP_505463.1  H2A          cH2A  cH2A_(Animals)           
EEC09557.1          EEC09557.1  H2A          cH2A  cH2A_(Animals)           
NP_724343.1        NP_724343.1  H2A          cH2A  cH2A_(Animals)           
XP_001119899.1  XP_001119899.1  H2A          cH2A  cH2A_(Animals)           
EDO48405.1          EDO48405.1  H2A          cH2A  cH2A_(Animals)           

                       gi ncbi_gene_id hgnc_gene_name taxonomy_id  \
NP_505463.1      17562014                                    6239   
EEC09557.1      215500063                                    6945   
NP_724343.1      24585673                                    7227   
XP_001119899.1  110764935                                    7460   
EDO48405.1      156227602                                   45351   

                               organism      phylum        class  \
NP_505463.1      Caenorhabditis elegans    Nematoda  Chromadorea   
EEC09557.1            Ixodes scapularis  Arthropoda    Arachnida   
NP_724343.1     Drosophila melanogaster  Arthropoda      Insecta   
XP_001119899.1           Apis mellifera  Arthropoda      Insecta   
EDO48405.1       Nematostella vectensis    Cnidaria     Anthozoa   

               taxonomy_group info            references  \
NP_505463.1                         26989147 22650316(?)   
EEC09557.1                          26989147 22650316(?)   
NP_724343.1                         26989147 22650316(?)   
XP_001119899.1                      26989147 22650316(?)   
EDO48405.1                          26989147 22650316(?)   

                                                         sequence  
NP_505463.1     MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...  
EEC09557.1      MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...  
NP_724343.1     MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...  
XP_001119899.1  MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...  
EDO48405.1      MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...

In [7]:
cs.data.shape

(561, 16)

In [8]:
h2bv_accessions = [
    "AAO24603.1",
]

In [13]:
df = pd.DataFrame(
    {
        "accession": h2bv_accessions,
        "type": ["H2B"] * len(h2bv_accessions),
        "variant_group": ["H2B.V"] * len(h2bv_accessions),
        "variant": ["H2B.V"] * len(h2bv_accessions),
        "references": ["16303849"] * len(h2bv_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [14]:
df.head()

accession type variant_group variant references
accession                                                   
AAO24603.1  AAO24603.1  H2B         H2B.V   H2B.V   16303849

In [15]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((562, 16), [])

In [16]:
cs.data.tail()

accession type variant_group                    variant  \
NP_005312.1  NP_005312.1   H1          H1.4   H1.4_(Homo_sapiens)__???   
NP_005311.1  NP_005311.1   H1          H1.3   H1.3_(Homo_sapiens)__???   
NP_006017.1  NP_006017.1   H1         H1.10  H1.10_(Homo_sapiens)__???   
NP_005309.1  NP_005309.1   H1          H1.0   H1.0_(Homo_sapiens)__???   
AAO24603.1    AAO24603.1  H2B         H2B.V                      H2B.V   

            doublet gi ncbi_gene_id hgnc_gene_name taxonomy_id      organism  \
NP_005312.1                  3008.0           H1-4        9606  Homo sapiens   
NP_005311.1                  3007.0           H1-3        9606  Homo sapiens   
NP_006017.1                  8971.0          H1-10        9606  Homo sapiens   
NP_005309.1                  3005.0           H1-0        9606  Homo sapiens   
AAO24603.1                                                                     

               phylum     class taxonomy_group info references  \
NP_005312.1  Chordata  Mammalia       Mammalia        26689747   
NP_005311.1  Chordata  Mammalia       Mammalia        26689747   
NP_006017.1  Chordata  Mammalia       Mammalia        26689747   
NP_005309.1  Chordata  Mammalia       Mammalia        26689747   
AAO24603.1                                            16303849   

                                                      sequence  
NP_005312.1  MSETAPAAPAAPAPAEKTPVKKKARKSAGAAKRKASGPPVSELITK...  
NP_005311.1  MSETAPLAPTIPAPAEKTPVKKKAKKAGATAGKRKASGPPVSELIT...  
NP_006017.1  MSVELEEALPVTTAEGMAKKVTKAGGSAALSPSKKRKNSKKKNQPG...  
NP_005309.1  MTENSTSAPAAKPKRAKASKKSTDHPKYSDMIVAAIQAEKNRAGSS...  
AAO24603.1

In [17]:
cs.data[cs.data["variant"] == "H2B.V"].shape

(1, 16)

In [18]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/l_singh/hdb/project_dir/histonedb/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [19]:
cs.data[cs.data["variant"] == "H2B.V"].shape

(1, 16)

In [20]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 5702
 changes to 5702
 changes to Trypanosoma brucei brucei
 changes to Euglenozoa
 changes to Kinetoplastea


In [21]:
cs.data[cs.data["variant"] == "H2B.V"].shape

(1, 16)

In [22]:
cs.data[cs.data["variant"] == "H2B.V"]

accession type variant_group variant doublet gi ncbi_gene_id  \
AAO24603.1  AAO24603.1  H2B         H2B.V   H2B.V                           

           hgnc_gene_name taxonomy_id                   organism      phylum  \
AAO24603.1                       5702  Trypanosoma brucei brucei  Euglenozoa   

                    class taxonomy_group info references sequence  
AAO24603.1  Kinetoplastea                       16303849

## Updating sequences for H2B.V

In [23]:
cs.data[cs.data['sequence'] == '']

accession type variant_group variant doublet gi ncbi_gene_id  \
AAO24603.1  AAO24603.1  H2B         H2B.V   H2B.V                           

           hgnc_gene_name taxonomy_id                   organism      phylum  \
AAO24603.1                       5702  Trypanosoma brucei brucei  Euglenozoa   

                    class taxonomy_group info references sequence  
AAO24603.1  Kinetoplastea                       16303849

In [24]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs


/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


Sequence for AAO24603.1 changes from  to MPPTKGGKRPLPLGGKGKGKRPPGQTTKSSSSRKKSGARRGKKQQRWDLYIHRTLRQVYKRGTLSKAAVRVLSSFIEDMYGKIQAEAVHVACINNVKTLTAREIQTSARLLLPPELAKHAMSEGTKAVAKYNASREEAYSKVL
Sequences updated: 1


In [25]:
cs.data[cs.data["variant"] == "H2B.V"]

accession type variant_group variant doublet gi ncbi_gene_id  \
AAO24603.1  AAO24603.1  H2B         H2B.V   H2B.V                           

           hgnc_gene_name taxonomy_id                   organism      phylum  \
AAO24603.1                       5702  Trypanosoma brucei brucei  Euglenozoa   

                    class taxonomy_group info references  \
AAO24603.1  Kinetoplastea                       16303849   

                                                     sequence  
AAO24603.1  MPPTKGGKRPLPLGGKGKGKRPPGQTTKSSSSRKKSGARRGKKQQR...

In [26]:
cs.save()

Added sequence with accession AAO24603.1
cp histones.csv backups/histones.csv-Mar0724180010
Previous data backuped to backups/histones.csv-Mar0724180010
Results saved to histones.csv
